In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import argparse
import logging
import time
from os.path import join
import sys
from hist.intervals import ratio_uncertainty
sys.path.append('../')

from coffea import processor, util

In [ ]:
from collections import defaultdict
import os
from os.path import join
import hist
import mplhep as hep
import seaborn as sns
from coffea import util
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_systematic, plot_m4l_systematic, get_category_labels, get_color_list
from azh_analysis.utils.parameters import get_lumis
import warnings
warnings.filterwarnings('ignore')
hep.style.use(["CMS", "fira", "firamath"])

# systematics to plot 
k_shifts = ["tauES", "efake", "mfake", "eleES", "eleSmear", "muES", "unclMET", "tauID_10"]
e_shifts = ["l1prefire", "pileup", "btag"]

# filename info
years = ["2016preVFP", "2016postVFP", "2017", "2018"]
lumi = get_lumis()[year]
source = "signal"
mass = "300"

# grab a specific systematic
syst = "unclMET"
plot = False
nom = util.load(f"../output_tighten_mtt/{source}_UL_{year}_all_OS.coffea")

cat_labels = get_category_labels()
colors = get_color_list()

outdir = f"../plots/systematics/{year}"
print(cat_labels)

In [ ]:
var = "m4l_binopt"
for cat, cat_label in cat_labels.items():
    for btag in [0]:      
        if "signal" in source:
            nom_s = sum([
                sum([v for k, v in util.load(f"../output_tighten_mtt/signal_UL_{year}_all_OS.coffea")[var].items() if (mass in k) and ("GluGlu" in k)]) 
                for year in years
            ])
            shift_s = sum([
                sum([v for k, v in util.load(f"../output_tighten_mtt/signal_UL_{year}_all_OS.coffea")[var].items() if (mass in k) and ("GluGlu" in k)]) 
                for year in years
            ])
        else:
            nom_s = sum([v for k, v in nom[var].items()])
            shift_s = sum([v for k, v in nom[var].items()])
        
        # group, cat, btag, syst, mass_type, :
        nom_s = nom_s[::sum, 'ee'+cat, ::sum, 'nom', :, :] + nom_s[::sum, 'mm'+cat, ::sum, 'nom', :, :]
        shift_up = shift_s[::sum, 'ee'+cat, ::sum, syst+"_up", :, :] + shift_s[::sum, 'mm'+cat, ::sum, syst+"_up", :, :]
        shift_down = shift_s[::sum, 'ee'+cat, ::sum, syst+"_down", :, :] + shift_s[::sum, 'mm'+cat, ::sum, syst+"_down", :, :]
        plot_m4l_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, mass_label=r"$m_{ll\tau\tau}$ [GeV]", 
            logscale=False,
            xlim=[0,500],
            year=year,
            lumi = lumi,
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "mtt"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, :, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, :, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, :, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, :, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, :, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, :, ::sum, syst+"_down", :]
        plot_m4l_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, mass_label=r"$m_{\tau\tau}$ [GeV]",
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "mll"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        # group, cat, sign, btag, syst, vals
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_down", :]
        plot_systematic(
            nom_s, shift_up, shift_down,
            syst, cat_label, r"$m_{ll}$ [GeV]", 
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf") if plot else None
        )

In [ ]:
var = "pt"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        for leg in ['3', '4']:
            nom_s = sum(nom[var].values())
            nom_s = nom_s[::sum, 'ee'+cat, 0, leg, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, leg, ::sum, 'nom', :]
            shift_s = sum(shift[var].values())
            shift_up = shift_s[::sum, 'ee'+cat, 0, leg, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, leg, ::sum, syst+"_up", :]
            shift_down = shift_s[::sum, 'ee'+cat, 0, leg, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, leg, ::sum, syst+"_down", :]
            plot_systematic(
                nom_s, shift_up, shift_down, 
                syst, cat_label, f"Leg {leg} " + r"$p_T$ [GeV]",
                year=int(year), lumi = lumis[year],
                outfile = join(outdir, f"{source}_{cat}_{syst}_{var}{leg}.pdf") if plot else None
            )

In [ ]:
var = "met"
for cat, cat_label in cat_labels.items():
    for bcat in [0]:
        nom_s = sum(nom[var].values())
        nom_s = nom_s[::sum, 'ee'+cat, 0, ::sum, 'nom', :] + nom_s[::sum, 'mm'+cat, 0, ::sum, 'nom', :]
        shift_s = sum(shift[var].values())
        shift_up = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_up", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_up", :]
        shift_down = shift_s[::sum, 'ee'+cat, 0, ::sum, syst+"_down", :] + shift_s[::sum, 'mm'+cat, 0, ::sum, syst+"_down", :]
        plot_systematic(
            nom_s, shift_up, shift_down, 
            syst, cat_label, r"$E_T^\mathrm{miss}$ [GeV]",
            year=int(year), lumi = lumis[year],
            outfile = join(outdir, f"{source}_{cat}_{syst}_{var}.pdf")
        )